In [11]:
#Create scenarios for PP location based on "aggregation" and "distance from outfall"
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
global str
from pyswmm import Simulation, Nodes, LidControls, Subcatchments, RainGages   

In [12]:
#function to assign the PP in the input files
def assign_clusters_input_file(df,file_name):
    
    #Prepare the text to be added in [LID USAGE]
    names=[]
    selection=[]
    PP_to_assign_last_cluster=df["area_ac"].iloc[-1]
    switch=0

    txt= ""

    for i in PP_ids: #the ID of each cluster in df
        selection=df_subcatch[(df_subcatch['clust100']==i)] #extracts the subcatchments of each cluster
        selection.columns = ["x", "y", "area","clust100"]
        #if it is the last cluster, the area to be assigned is not all the area of the subctachment
        if i == PP_ids[-1]:        
            for row in selection.iterrows():     
                #if the subcatchment area is larger than the PP area to assign, then use all the subcatch area
                if row[1][2]> PP_to_assign_last_cluster:
                    if PP_to_assign_last_cluster>0.01: #while there is still PP area to asign
                        txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(PP_to_assign_last_cluster*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"
                        PP_to_assign_last_cluster=0
                    break
                #otherwise, assign the PP that is left
                else:            
                    txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(row[1][2]*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"
                    PP_to_assign_last_cluster=PP_to_assign_last_cluster-math.floor(row[1][2]*43560*100)/100

        #otherwise, assign all the subcatchment area to PP
        else:
            for row in selection.iterrows():  
                txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(row[1][2]*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"

    #Identifies the correct location to inject the Text in the input file.
    with open('BSA_Main_Model.inp') as f:
        counter=1
        datafile = f.readlines()
        for line in datafile:
            if '[LID_USAGE]' in line:
                SUBC=counter
            counter=counter+1    

    NEWfile = open(file_name, "w")
    LINEcounter=1       
    for line in datafile:
        NEWfile.write(line)
        if LINEcounter==SUBC+2: #two lines ahead [LID_USAGE], making space for headers
            NEWfile.write(txt)
        LINEcounter=LINEcounter+1

In [56]:
#load required files

#load subcatchments ids for each cluster
subcatch_TEMP= pd.read_csv('cso014_cluster_elements.txt', sep="\t")
subcatch_NAMES=subcatch_TEMP['NAME']
data_subcatch= np.genfromtxt(fname='cso014_cluster_elements.txt')
data_subcatch=np.delete(data_subcatch, 0, axis=0)
data_subcatch=np.delete(data_subcatch, 0, axis=1)
df_subcatch = pd.DataFrame(data_subcatch)
df_subcatch.columns = ["x", "y", "area","clust100"]
df_subcatch.index=subcatch_NAMES
#print(df_subcatch.loc['Parking_E-MH-7650',:])
#print(df_subcatch[df_subcatch['clust100']==10])

#load clusters list and sector
data = np.genfromtxt(fname='cso014_CLUSTERS_S2.txt')
data=np.delete(data, 0, axis=0)
df = pd.DataFrame(data)
df.columns = ["cluster100", "x", "y", "distance_outfall","rank_dist","area", "sector"]


In [72]:
clusters_number=100 #number of clusters
iterations=3 #number of iterations= number of runs
random.seed(1234)
nsect=3 #number of sectors 


for i in range(1,nsect+1): #iterating over the sectors
    for j in range(iterations): #number of iterations per sector
        PP_assigned=[] #create an empty list to store the PP area assigned
        PP_ids=[] #create an empty list to store the subcatchments id's to which PP area is assigned
        PP_to_assign=50 #area in acres to be assigned
        df_temp=df[df['sector']==i] #extract a temporal dataframe with clusters in the sector i
        listclusters=list(df_temp.cluster100) #transform to a list
        
        while PP_to_assign>0: #while there is still PP are to asign
            current_clust=listclusters[random.randint(0, len(listclusters)-1)] #pick a cluster randomly
            listclusters.remove(current_clust) #remove the selected cluster from the list (to avoid repetition)
            #if the subcatchment area is larger than the PP area to assign, then use all the subcatch area
            area_cl=float(df_temp[df_temp["cluster100"]==current_clust].area) #
            if area_cl> PP_to_assign:
                PP_assigned.append(PP_to_assign)
                PP_to_assign=0
            #otherwise, assign the PP that is left
            else:            
                PP_to_assign= PP_to_assign-area_cl
                PP_assigned.append(area_cl)
            PP_ids.append(current_clust)
        df3= pd.DataFrame(list(zip(PP_ids, PP_assigned)), columns =['cluster', 'area_ac'])
        print(df3) 

        file_name='sector_'+str(i)+'_it_'+str(j)+'.inp'
        #Run the function to create a new input file
        assign_clusters_input_file(df3,file_name)

        #Testing the file is correct
        with Simulation(file_name) as sim:
            for raingage in RainGages(sim):
                #print(raingage)
                print(raingage.raingageid) 
        

   cluster  area_ac
0     79.0    7.662
1     39.0    4.257
2     69.0    6.722
3     53.0   11.676
4     70.0    4.288
5     40.0   10.716
6     31.0    4.679
0.9in_24hr
   cluster  area_ac
0     30.0    9.368
1     79.0    7.662
2     69.0    6.722
3     31.0    7.373
4     39.0    4.257
5     54.0    5.405
6     53.0    9.213
0.9in_24hr
   cluster  area_ac
0     53.0   11.676
1     98.0    4.121
2     69.0    6.722
3     39.0    4.257
4     40.0   10.716
5     79.0    7.662
6     30.0    4.846
0.9in_24hr
   cluster  area_ac
0     64.0    8.856
1     39.0    4.257
2     31.0    7.373
3     69.0    6.722
4      4.0   14.187
5     70.0    4.288
6     53.0    4.317
0.9in_24hr
   cluster  area_ac
0     15.0    6.114
1     39.0    4.257
2     70.0    4.288
3     69.0    6.722
4     40.0   10.716
5     79.0    7.662
6      8.0    7.381
7     31.0    2.860
0.9in_24hr
   cluster  area_ac
0     64.0    8.856
1      8.0    7.381
2     69.0    6.722
3     30.0    9.368
4     98.0    4.121
5    

In [ ]:
#create inp files for "LOW" distance, "HIGH" aggregation
distance=['low','medium','high']
aggregation=['low','medium','high']
clusters_number=100
iterations=100
PP_tot_area=50

for k in range(iterations):
    for i in distance:
        for j in aggregation:
            first_cluster=random.randint(0, clusters_number-1)
            if df.iloc[first_cluster]['area']< PP_tot_area:
                print("True")
                PP_left=PP_tot_area-df.iloc[first_cluster]['area']
            else:
                print("False")
    

In [ ]:
####Split all area in same cluster

PP_assigned=[]
PP_ids=[]
first_cluster=random.randint(0, clusters_number-1)
PP_to_assign=50

while PP_to_assign>0:
    print(PP_to_assign)
    if df.iloc[first_cluster]['area']> PP_to_assign:
        PP_assigned.append(PP_to_assign)
        PP_to_assign=0
        assign_PP_to_subcatch(PP_to_assign,first_cluster)
    else:
        PP_to_assign=PP_to_assign-df.iloc[first_cluster]['area']
        PP_assigned.append(df.iloc[first_cluster]['area'])
        
    PP_ids.append(first_cluster)
    first_cluster=random.randint(0, clusters_number-1)
    while first_cluster in PP_ids:
        first_cluster=random.randint(0, clusters_number-1)
        
df3= pd.DataFrame(list(zip(PP_ids, PP_assigned)), columns =['cluster', 'area_ac']) 